In [0]:
!pip install -U tensorflow

In [0]:
from google.colab import files
import zipfile
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import tensorflow_datasets as tfds
from shutil import copyfile

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [0]:
chk = os.listdir('.')
print(chk)

In [0]:
!rm 'dataset.zip'

In [0]:
uploaded = files.upload()

In [0]:
zpf = zipfile.ZipFile("dataset.zip")
zpf.extractall()
zpf.close()

In [0]:
!mkdir cells
!mkdir texts
!mkdir texts/images
!mkdir labels
!mkdir labels/files

In [0]:
!rm -r cells

In [0]:
print(os.listdir('dataset'))

In [0]:
sets = os.listdir('dataset')
for s in sets:
  set_dir = 'dataset/' + s
  files = os.listdir(set_dir)
  for f in files:
    if 'cell_' in f:
      copyfile(set_dir + '/' + f, 'cells/' + f)

In [0]:
files = os.listdir('cells/')
for f in files:
  if '.png' in f:
    copyfile( 'cells/' + f, 'texts/images/' + f)
  else:
    copyfile( 'cells/' + f, 'labels/files/' + f)

In [0]:
print(os.listdir('labels/files/'))

In [0]:
files = os.listdir('labels/files')
files.sort()
labels = []
for f in files:
  with open('labels/files/'+f) as opnd:
    txt = opnd.read()
  txt = txt[1:-1]
  labels.append(txt)

In [0]:
# Kopia żywcem z zajęć (z braku czasu). Chodzi tylko o encoder zawarty w 'info'.
(train_data, test_data), info = tfds.load(
    # Use the version pre-encoded with an ~8k vocabulary.
    'imdb_reviews/subwords8k', 
    # Return the train/test datasets as a tuple.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Return (example, label) pairs from the dataset (instead of a dictionary).
    as_supervised=True,
    # Also return the `info` structure. 
    with_info=True)

In [0]:
encoder = info.features['text'].encoder

In [0]:
encoded_text = [encoder.encode(w) for w in labels]
print(encoded_text)

In [0]:
files = os.listdir('texts/images')
files.sort()
images = []
for f in files:
  cont = tf.io.read_file('texts/images/'+f);
  arr = tf.io.decode_png(cont);
  images.append(arr)

In [0]:
# Sprawdziłem że największa długość labels to 9.
def pad(t):
  l = len(t)
  for _ in range(10-l):
    t.append(-1)

In [0]:
def depad_ret(t):
  i = 0
  l = len(t)
  while i < l:
    if t[i] < 0:
      break
    i+=1
  return t[:i]

In [0]:
for t in encoded_text:
  pad(t)

In [0]:
print(encoded_text)

In [0]:
l = len(images)
pairs = [(np.array(images[i]), np.array(encoded_text[i])) for i in range(l)]
#pairs = np.array(pairs)

In [0]:
print(encoder.decode(depad_ret(pairs[0][1])))

In [0]:
array_to_img(pairs[0][0])

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((pairs[:][0], pairs[:][1]))

In [0]:
model = keras.Sequential([
  keras.layers.Conv2D(filters=32, kernel_size=23, padding='same'),
  keras.layers.MaxPooling2D((2, 2)),
  keras.layers.Conv2D(filters=32, kernel_size=23, padding='same'),
  keras.layers.MaxPooling2D((2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(512),
  keras.layers.GRU(512, return_sequences=True, kernel_initializer='he_normal'),
  keras.layers.GRU(128, return_sequences=True, kernel_initializer='he_normal'),
  keras.layers.GRU(32, return_sequences=True, kernel_initializer='he_normal'),
  keras.layers.GRU(10, return_sequences=True, kernel_initializer='he_normal'),
])

In [0]:
sgd = keras.optimizers.SGD(learning_rate=0.02,
  decay=1e-6,
  momentum=0.9,
  nesterov=True)

In [0]:
 model.compile(loss='mae', optimizer='adam')

In [0]:
his = model.fit(
    train_dataset,
    steps_per_epoch=10,
    epochs=1,
    validation_data=train_dataset,
    validation_steps=5)